In [1]:
import os
import logging
import numpy as np
import pandas as pd
import torch
import scipy.io as sio
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
from scipy.stats import zscore

# -------------------------------------------------------------------
# 1) PATHS AND CSV LOADING
# -------------------------------------------------------------------
base_dir = "/home/santiago/Desktop/GrandMeanNorm/"
csv_path = os.path.join(base_dir, "SubjectsDataAndTests.csv")
roi_dir = os.path.join(base_dir, "ROISignals")
qc_dir = os.path.join(base_dir, "QualityChecks")
os.makedirs(qc_dir, exist_ok=True)

summary_path = os.path.join(qc_dir, "summary_statistics.csv")

# Load metadata
subjects_df = pd.read_csv(csv_path)
subject_ids = subjects_df["SubjectID"].astype(str).tolist()

print(f"Total subjects to process: {len(subject_ids)}\n")

# -------------------------------------------------------------------
# 2) LOGGING CONFIG
# -------------------------------------------------------------------
logging.basicConfig(filename='qc_processing.log', level=logging.ERROR)

# -------------------------------------------------------------------
# 3) FUNCTIONS
# -------------------------------------------------------------------

def check_and_visualize_signals(signals, subject_id, save_dir):
    """Generates and saves quality control plots for a subject."""
    num_timepoints, num_rois = signals.shape

    fig, axs = plt.subplots(2, 2, figsize=(14, 10))

    # Heatmap
    sns.heatmap(signals.T, cmap="viridis", ax=axs[0, 0], cbar_kws={"label": "Amplitude"})
    axs[0, 0].set_title(f"Subject {subject_id} - Signal Heatmap")
    axs[0, 0].set_ylabel("ROI")
    axs[0, 0].set_xlabel("Time points")

    # Amplitude Histogram
    axs[0, 1].hist(signals.flatten(), bins=50, color="royalblue", edgecolor="black")
    axs[0, 1].set_title("Amplitude Distribution")
    axs[0, 1].set_xlabel("Amplitude")
    axs[0, 1].set_ylabel("Frequency")

    # Mean signal
    mean_signal = np.mean(signals, axis=1)
    axs[1, 0].plot(mean_signal, color="darkorange")
    axs[1, 0].set_title("Mean Signal Across ROIs")
    axs[1, 0].set_xlabel("Time points")
    axs[1, 0].set_ylabel("Mean amplitude")

    # Std signal
    std_signal = np.std(signals, axis=1)
    axs[1, 1].plot(std_signal, color="seagreen")
    axs[1, 1].set_title("Standard Deviation Across ROIs")
    axs[1, 1].set_xlabel("Time points")
    axs[1, 1].set_ylabel("Std amplitude")

    plt.tight_layout()

    save_path = os.path.join(save_dir, f"subject_{subject_id}_qc.png")
    plt.savefig(save_path)
    plt.close()

# -------------------------------------------------------------------
# 4) MAIN PROCESSING LOOP
# -------------------------------------------------------------------
summary_records = []

with tqdm(total=len(subject_ids), desc="Processing subjects") as pbar:
    for subject_id in subject_ids:
        mat_filename = f"ROISignals_{subject_id}.mat"
        mat_filepath = os.path.join(roi_dir, mat_filename)

        if not os.path.exists(mat_filepath):
            print(f"File not found: {mat_filepath}")
            pbar.update(1)
            continue

        try:
            data = sio.loadmat(mat_filepath)
        except Exception as e:
            logging.error(f"Error loading {mat_filepath}: {e}")
            pbar.update(1)
            continue

        if "ROISignals" in data:
            signals = data["ROISignals"]
        elif "signals" in data:
            signals = data["signals"]
        else:
            print(f"Skipping {mat_filepath}: no 'ROISignals' or 'signals' key.")
            pbar.update(1)
            continue

        # Detectar ROIs con NaNs
        nan_per_roi = np.isnan(signals).sum(axis=0)  # cantidad de NaNs por ROI
        inf_per_roi = np.isinf(signals).sum(axis=0)  # cantidad de infinitos por ROI

        # Mostrar las ROIs que tienen NaNs o Infs
        problematic_rois = np.where((nan_per_roi > 0) | (inf_per_roi > 0))[0]

        if len(problematic_rois) > 0:
            print(f"⚠️ Subject {subject_id} — ROIs with NaNs or Infs: {problematic_rois.tolist()}")


        has_nan_inf = np.isnan(signals).any() or np.isinf(signals).any()
        if has_nan_inf:
            print(f"⚠️ Warning: {mat_filepath} contains NaN/Inf. Not modifying the file.")


        # Basic stats
        num_timepoints, num_rois = signals.shape
        mean_val = np.mean(signals)
        std_val = np.std(signals)
        min_val = np.min(signals)
        max_val = np.max(signals)

        # Additional QC metrics
        zero_counts = np.sum(signals == 0, axis=0)  # zeros per ROI
        coef_var = np.std(signals, axis=0) / (np.mean(signals, axis=0) + 1e-8)  # coef var per ROI

        # Visual QC
        check_and_visualize_signals(signals, subject_id, qc_dir)

        # Record summary
        summary_records.append({
            "SubjectID": subject_id,
            "NumTimepoints": num_timepoints,
            "MeanAmplitude": mean_val,
            "StdAmplitude": std_val,
            "MinAmplitude": min_val,
            "MaxAmplitude": max_val,
            "ZeroPercROIs": np.mean(zero_counts / num_timepoints),
            "MeanCoefVarROIs": np.mean(coef_var),
            "HasNaN_Inf": int(has_nan_inf)  # 0 = no tiene, 1 = tiene NaN/Inf
        })

        pbar.update(1)

# -------------------------------------------------------------------
# 5) SAVE SUMMARY CSV
# -------------------------------------------------------------------
summary_df = pd.DataFrame(summary_records)
summary_df.to_csv(summary_path, index=False)
print(f"\n✅ Summary statistics saved to: {summary_path}")

# -------------------------------------------------------------------
# 6) HISTOGRAM OF TIMEPOINTS
# -------------------------------------------------------------------
plt.figure(figsize=(8, 6))
plt.hist(summary_df["NumTimepoints"], bins=20, color="slateblue", edgecolor="black")
plt.title("Histogram of Timepoints Across Subjects")
plt.xlabel("Number of Timepoints")
plt.ylabel("Number of Subjects")
plt.grid(True)
plt.tight_layout()

hist_path = os.path.join(qc_dir, "histogram_timepoints.png")
plt.savefig(hist_path)
plt.close()

print(f"✅ Timepoints histogram saved to: {hist_path}")


# -------------------------------------------------------------------
# 7) SCATTERPLOT: Number of Timepoints vs. Percentage of Zero ROIs
# -------------------------------------------------------------------
plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=summary_df,
    x="NumTimepoints",
    y="ZeroPercROIs",
    s=60,  # Tamaño de los puntos
    color="crimson",
    edgecolor="black"
)
plt.title("Timepoints vs. Percentage of Zero ROIs")
plt.xlabel("Number of Timepoints")
plt.ylabel("Percentage of Zero ROIs")
plt.grid(True)
plt.tight_layout()

scatter_path = os.path.join(qc_dir, "scatter_timepoints_vs_zeros.png")
plt.savefig(scatter_path)
plt.close()

print(f"✅ Scatterplot saved to: {scatter_path}")



Total subjects to process: 432



Processing subjects:   0%|          | 0/432 [00:00<?, ?it/s]

⚠️ Warning: /home/santiago/Desktop/GrandMeanNorm/ROISignals/ROISignals_002_S_0295.mat contains NaN/Inf. Not modifying the file.


Processing subjects:   0%|          | 1/432 [00:00<06:03,  1.19it/s]

⚠️ Warning: /home/santiago/Desktop/GrandMeanNorm/ROISignals/ROISignals_002_S_0413.mat contains NaN/Inf. Not modifying the file.


...

⚠️ Warning: /home/santiago/Desktop/GrandMeanNorm/ROISignals/ROISignals_941_S_4036.mat contains NaN/Inf. Not modifying the file.


Processing subjects: 100%|█████████▉| 431/432 [05:45<00:00,  1.23it/s]

⚠️ Warning: /home/santiago/Desktop/GrandMeanNorm/ROISignals/ROISignals_941_S_4187.mat contains NaN/Inf. Not modifying the file.


Processing subjects: 100%|██████████| 432/432 [05:46<00:00,  1.25it/s]



✅ Summary statistics saved to: /home/santiago/Desktop/GrandMeanNorm/QualityChecks/summary_statistics.csv
✅ Timepoints histogram saved to: /home/santiago/Desktop/GrandMeanNorm/QualityChecks/histogram_timepoints.png
✅ Scatterplot saved to: /home/santiago/Desktop/GrandMeanNorm/QualityChecks/scatter_timepoints_vs_zeros.png
